<html>
    <div dir="rtl">
        این نوتبوک، روی دیتاست میراث جملات را بررسی می‌کند و ترتیب اجزاء جمله را بسته به POS tag شان یاد می‌گیرد.
        از این طریق می‌تواند بهترین جایگشت‌های ممکن برای کلمات را پیدا کند.
    </div>
</html>

In [14]:
%pip install hazm

Note: you may need to restart the kernel to use updated packages.


In [7]:
from hazm import POSTagger, Normalizer

tagger = POSTagger(model="resources/pos_tagger.model")
text = "رفته بودم خونه ی دوستم"

normalized_text = Normalizer().normalize(text)
tagged_text = tagger.tag(normalized_text.split())
print(tagged_text)

[('رفته', 'VERB'), ('بودم', 'VERB'), ('خونه\u200cی', 'NOUN,EZ'), ('دوستم', 'NOUN')]


In [18]:
from hazm import SentenceTokenizer, WordTokenizer
from tqdm import tqdm

with open('resources/Datasets/MirasText_sample.txt', 'r') as f:
    text = f.read()
    sentences = SentenceTokenizer().tokenize(text)

"""
Let's use a matrix to store the sequence orders for each type of sentence
first dimension: number of verbs
second dimension: number of nouns
third dimension: number of adjectives
...
"""

the_set = set()

for sentece in tqdm(sentences[:2446]):
    words = WordTokenizer().tokenize(sentece)
    tagged_words = tagger.tag(words)
    for word, tag in tagged_words:
        the_set.add(tag)

print(the_set)



  1%|▏         | 35/2446 [00:02<03:20, 12.00it/s]


KeyboardInterrupt: 

In [55]:
from enum import Enum, auto

"""
each sentence will be associated with a vector of 16 elements, each element representing the number of words with that tag in the sentence
(N, Ne, V, Aj, Adv, Det, P, Conj, Pron, Num, Postp, Prep, Interj, Abbrev, Aux, Punc)
example: بابا آب داد
(2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0)
"""

class Tag(Enum):
    # starting with zero
    NOUN = auto()
    NE = auto()
    VERB = auto()
    ADJ = auto()
    ADV = auto()
    ADP = auto()
    DET = auto()
    P = auto()
    CONJ = auto()
    PRON = auto()
    NUM = auto()
    POSTP = auto()
    PREP = auto()
    INTERJ = auto()
    ABBREV = auto()
    AUX = auto()
    PUNCT = auto()
    SCONJ = auto()
    CCONJ = auto()




def tag_dataset():
    with open('resources/Datasets/MirasText_sample.txt', 'r') as f:
        text = f.read()
    sentences = SentenceTokenizer().tokenize(text)
    dataset = {}
    for sentece in tqdm(sentences[:24]):
        words = WordTokenizer().tokenize(sentece)
        tagged_words = tagger.tag(words)

        sentence_vector = [0] * len(Tag)
        for word, tag in tagged_words:
            pure_tag = tag.split(',')[0] # some tags are followed by ,EZ so we ignore it!
            tag_index = Tag[pure_tag].value - 1
            sentence_vector[tag_index] += 1
        dataset[tuple(sentence_vector)] = [tag.split(',')[0] for _, tag in tagged_words]

    return dataset

tagged_dataset = tag_dataset()

100%|██████████| 24/24 [00:02<00:00, 11.85it/s]


<html>
    <div dir="rtl">
        کاری که دوست داریم انجام بدهیم، این است که بتوانیم جملات را بر اساس اجزاء جمله‌شان دسته‌بندی کنیم.
        اما اگر بخواهیم به صورت دقیق تمام اجزاء را در نظر بگیریم کمی دشوار خواهد بود و به دیتاست خیلی بزرگ‌تری نیاز خواهیم داشت.
        برای همین باید بتوانیم کمی دسته‌بندی‌ها را جامع‌تر کنیم تا دسته‌های مختلف را در بر بگیرد.
        برای همین، برخی از اجزاء جمله که توسط هضم دسته‌بندی شده‌اند را بکسان در نظر می‌گیریم.
    </div>
</html>

In [56]:
tagged_dataset

{(3, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0): ['NOUN',
  'ADV',
  'ADP',
  'NOUN',
  'NOUN',
  'VERB',
  'PUNCT'],
 (20, 0, 4, 5, 0, 7, 0, 0, 0, 1, 6, 0, 0, 0, 0, 0, 3, 3, 0): ['NOUN',
  'NOUN',
  'PUNCT',
  'NUM',
  'NUM',
  'NOUN',
  'NUM',
  'PUNCT',
  'NUM',
  'NOUN',
  'NOUN',
  'ADJ',
  'ADP',
  'NOUN',
  'NUM',
  'ADJ',
  'PRON',
  'VERB',
  'SCONJ',
  'NOUN',
  'NOUN',
  'ADP',
  'NOUN',
  'ADP',
  'NOUN',
  'ADJ',
  'ADP',
  'NOUN',
  'VERB',
  'SCONJ',
  'NOUN',
  'ADP',
  'NOUN',
  'ADJ',
  'ADP',
  'NOUN',
  'ADJ',
  'NOUN',
  'NOUN',
  'NOUN',
  'VERB',
  'SCONJ',
  'NOUN',
  'ADP',
  'NOUN',
  'NUM',
  'NOUN',
  'VERB',
  'PUNCT'],
 (18, 0, 4, 5, 0, 7, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 2, 3, 0): ['NUM',
  'NOUN',
  'PUNCT',
  'NOUN',
  'NOUN',
  'ADJ',
  'ADP',
  'NOUN',
  'NUM',
  'ADJ',
  'PRON',
  'VERB',
  'SCONJ',
  'NOUN',
  'NOUN',
  'ADP',
  'NOUN',
  'ADP',
  'NOUN',
  'ADJ',
  'ADP',
  'NOUN',
  'VERB',
  'SCONJ',
  'NOUN',
  'ADP',
  'NOUN',
  'ADJ',


In [60]:
with open('resources/Datasets/MirasText_sample.txt', 'r') as f:
    text = f.read()
for i in SentenceTokenizer().tokenize(text)[:24]:
    print(i)

NameError: name 'senteces' is not defined

In [26]:
print(Tag['N'].value)

0
